# Exploratory notebook for IMACLIM within MATMat-Trade

## Creating the MatMat-Trade model base on IMACLIM regions and sectors but on EXIOBASE 3 data

In [1]:
# make the necessary imports
import tqdm
import shutil
import warnings
import pandas as pd
import numpy as np
import multiprocessing


from main import Model
from src.utils import footprint_extractor
from src.scenarios import DICT_SCENARIOS
from src.advance import extract_data

In [2]:
# for some reason saved model has issues with the names of regions so we rebuild the model each times


shutil.rmtree('data/models/2015__pxp__IMACLIM_format__ghg')

In [3]:


model = Model(aggregation_name="IMACLIM_format")

#ignore performace warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

Loading data... (may take a few minutes)
Data loaded successfully !


## Creating the scenarios based on IMACLIM scenarios and the projected changes 

In [4]:

scenario_list=["INDC","INDC2020_1000","INDC2030_1000","INDC2020_1600","INDC2020_400","INDC2030_1600","baseline"]

for scenario in tqdm.tqdm(scenario_list[:1]):
    # create 2050 version of each scenarios applying emissivity changes and then technological changes
    model.new_counterfactual("IMACLIM_all_2050_"+scenario, DICT_SCENARIOS["emissivity_IMACLIM"], year=2050,scenario=scenario)
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=2050,scenario=scenario)
    # create the 2050 version based on production changes observed in IMACLIM (since we work on groos output, we need a reference to apply relative change : x_ref), with name new_name
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["production_change_IMACLIM"], year=2050,scenario="INDC",new_name="IMACLIM_all_prod_change_2050_"+scenario,x_ref=model.iot.x.sort_index())
    
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["consumption_change_imaclim"], year=2050,scenario="INDC",new_name="IMACLIM_all_cons_change_2050_"+scenario)

  0%|          | 0/1 [00:02<?, ?it/s]


KeyboardInterrupt: 

## Getting the emissions per scenario 

### Single process version, might be long to run 

In [ ]:

scenario_list=["INDC","INDC2020_1000","INDC2030_1000","INDC2020_1600","INDC2020_400","INDC2030_1600","baseline"]

start_year=pd.concat([ pd.concat([pd.DataFrame(footprint_extractor(model,region=region),index=[2015]).T  for region in model.regions],
                      keys=model.regions,
                      names=("regions","types")) for scenario in scenario_list],
          keys=scenario_list,
          names=("scenario","regions","types"))

Emissions=pd.concat([start_year for i in range(4)],axis=1,keys=["2015","2050","2050-Prod","2050-Cons"]).sort_index()
Emissions.columns=["2015","2050","2050-Prod","2050-Cons"]

production_based_emissions=pd. concat([ pd.concat([ model.iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"]for i in range(4)],axis=1,keys=["2015","2050","2050-Prod","2050-Cons"]) for scenario in scenario_list],
                                        keys=scenario_list,
                                        names=("scenario","region","sector"),
                                        axis=0).sort_index()


for scenario in tqdm.tqdm(scenario_list):
    # create 2050 version of each scenarios applying emissivity changes and then technological changes
    model.new_counterfactual("IMACLIM_all_2050_"+scenario, DICT_SCENARIOS["emissivity_IMACLIM"], year=2050,scenario=scenario)
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=2050,scenario=scenario)
    # create the 2050 version based on production changes observed in IMACLIM (since we work on groos output, we need a reference to apply relative change : x_ref), with name new_name
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["production_change_IMACLIM"], year=2050,scenario=scenario,new_name="IMACLIM_all_prod_change_2050_"+scenario,x_ref=model.iot.x.sort_index())
    
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["consumption_change_imaclim"], year=2050,scenario=scenario,new_name="IMACLIM_all_cons_change_2050_"+scenario)
    
    Emissions.loc[(scenario),"2050"]=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    Emissions.loc[(scenario),"2050-Prod"]=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    Emissions.loc[(scenario),"2050-Cons"]=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    production_based_emissions.loc[(scenario),"2050-Cons"]=model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    production_based_emissions.loc[(scenario),"2050-Prod"]=model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    production_based_emissions.loc[(scenario),"2050"]=model.counterfactuals["IMACLIM_all_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    del(model.counterfactuals["IMACLIM_all_2050_"+scenario])
    del(model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario])
    del(model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario])
    
    

### Multiple Process versio, should be faster

In [4]:

NBR_PROCESS=4

scenario_list=["INDC","INDC2020_1000","INDC2030_1000","INDC2020_1600","INDC2020_400","INDC2030_1600","baseline"]

start_year=pd.concat([ pd.concat([pd.DataFrame(footprint_extractor(model,region=region),index=[2015]).T  for region in model.regions],
                      keys=model.regions,
                      names=("regions","types")) for scenario in scenario_list],
          keys=scenario_list,
          names=("scenario","regions","types"))

Emissions=pd.concat([start_year for i in range(4)],axis=1,keys=["2015","2050","2050-Prod","2050-Cons"]).sort_index()
Emissions.columns=["2015","2050","2050-Prod","2050-Cons"]

production_based_emissions=pd. concat([ pd.concat([ model.iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"]for i in range(4)],axis=1,keys=["2015","2050","2050-Prod","2050-Cons"]) for scenario in scenario_list],
                                        keys=scenario_list,
                                        names=("scenario","region","sector"),
                                        axis=0).sort_index()


def task(scenario):
    model.new_counterfactual("IMACLIM_all_2050_"+scenario, DICT_SCENARIOS["emissivity_IMACLIM"], year=2050,scenario=scenario)
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=2050,scenario=scenario)
    # create the 2050 version based on production changes observed in IMACLIM (since we work on groos output, we need a reference to apply relative change : x_ref), with name new_name
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["production_change_IMACLIM"], year=2050,scenario=scenario,new_name="IMACLIM_all_prod_change_2050_"+scenario,x_ref=model.iot.x.sort_index())
    
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["consumption_change_imaclim"], year=2050,scenario=scenario,new_name="IMACLIM_all_cons_change_2050_"+scenario)
    
    ret1=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    ret2=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    ret3=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    ret4=model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    ret5=model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    ret6=model.counterfactuals["IMACLIM_all_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    del(model.counterfactuals["IMACLIM_all_2050_"+scenario])
    del(model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario])
    del(model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario])
    
    return ret1,ret2,ret3,ret4,ret5,ret6


pool = multiprocessing.Pool(processes=NBR_PROCESS)
result_list=pool.map(task,scenario_list)

for i in range(len(scenario_list)):
    scenario=scenario_list[i]
    Emissions.loc[(scenario),"2050"],Emissions.loc[(scenario),"2050-Prod"],Emissions.loc[(scenario),"2050-Cons"],production_based_emissions.loc[(scenario),"2050-Cons"],production_based_emissions.loc[(scenario),"2050-Prod"],production_based_emissions.loc[(scenario),"2050"]=result_list[i]
        
    

Problems on sums of columns  region  sector               
AFR     Terrestrial transport     333.209566
MDE     Terrestrial transport    4486.348277
RAL     Terrestrial transport     172.151602
dtype: float64
Problems on sums of columns  region  sector               
AFR     Terrestrial transport     47.043673
MDE     Terrestrial transport    146.430499
RAL     Terrestrial transport     25.875496
dtype: float64
Problems on sums of columns  region  sector               
AFR     Terrestrial transport     237.323465
MDE     Terrestrial transport    3148.991362
RAL     Terrestrial transport     124.123202
dtype: float64
Problems on sums of columns  region  sector               
AFR     Terrestrial transport     169.042045
MDE     Terrestrial transport    2024.383010
RAL     Terrestrial transport      89.719931
dtype: float64
Problems on sums of columns  region  sector               
AFR     Terrestrial transport     282.256477
MDE     Terrestrial transport    6422.740342
RAL     Terrestria

## Exploring the results

In [5]:
#adding IMACLIM direct emissions to the df
IMACLIM_emissions=extract_data(aggregation="IMACLIM_format")[6]

production_based_emissions["2050-IMACLIM"]=IMACLIM_emissions.drop("World",level=1)[2050].rename_axis(index=str.lower)*1e9
production_based_emissions["2015-IMACLIM"]=IMACLIM_emissions.drop("World",level=1)[2015].rename_axis(index=str.lower)*1e9

In [6]:
diff_rapport =((production_based_emissions["2050-IMACLIM"]/production_based_emissions["2015-IMACLIM"])-(production_based_emissions["2050-Prod"]/production_based_emissions["2015"]))

for scenario in scenario_list:
    print(scenario,diff_rapport.loc[scenario].max())

INDC 8.268941087408166e-12
INDC2020_1000 8.035572207631958e-12
INDC2030_1000 7.099210108663101e-12
INDC2020_1600 8.144818153255073e-12
INDC2020_400 6.13709083552294e-12
INDC2030_1600 8.04800670550776e-12
baseline 9.425571434462654e-12


In [7]:
Relative_error_2015=production_based_emissions.groupby(["scenario","region","sector"]).sum().loc["baseline"] 
Relative_error_2015=(Relative_error_2015["2015"]-Relative_error_2015["2015-IMACLIM"])/np.max([Relative_error_2015["2015"],Relative_error_2015["2015-IMACLIM"]],axis=0)
Relative_error_2015=Relative_error_2015*100
pd.concat([Relative_error_2015.loc[region] for region in Relative_error_2015.index.get_level_values("region").unique()],keys=Relative_error_2015.index.get_level_values("region").unique(),axis=1)

region,AFR,BRA,CAN,CHN,CIS,EUR,IND,JAN,MDE,RAL,RAS,USA
sector,,,,,,,,,,,,
Agriculture,-43.590339,37.134146,15.936325,-44.977011,-79.050224,-7.480474,48.257791,-60.567556,-55.777698,-18.200986,30.372903,-3.244593
Air,-45.142647,-22.186058,12.029989,71.750897,-2.496690,22.254563,46.981273,-44.709538,47.393486,-67.742879,39.151737,-7.363344
Coal,37.069602,43.351854,-88.326434,4.551393,-52.544245,-51.589768,96.477647,-35.200343,100.000000,100.000000,99.602004,-67.145125
Construction,84.523051,99.707779,76.583065,96.611540,71.457156,94.403003,96.048193,72.736079,83.043970,98.905166,96.741961,89.447613
Elec,-39.063432,72.813771,-61.749840,38.125084,-96.819867,-56.260718,27.300270,-32.720686,16.044896,-21.715810,5.708682,-39.344545
Gas,50.168897,81.877660,8.406563,97.880386,4.636883,87.340254,68.934163,100.000000,-13.784912,-2.699496,49.584495,71.617184
Industry,-43.194885,5.480878,-51.361436,21.771290,-72.690944,-30.088759,-9.858828,-60.712734,38.940292,9.302386,11.266783,-54.420385
Liquid fuels,-91.607445,-76.416717,-38.014303,-44.130341,-55.153795,-77.602159,-2.684537,-82.068727,100.000000,100.000000,-61.019781,99.869409
Maritime,-89.620191,-69.921130,46.180283,70.196181,100.000000,41.037293,93.972451,40.129431,-24.836034,-70.236938,-23.554738,-42.914006


In [8]:
Relative_error_2015=production_based_emissions.groupby(["scenario","region","sector"]).sum().loc["baseline"] 
Relative_error_2015=(Relative_error_2015["2050-Prod"]-Relative_error_2015["2050-IMACLIM"])/np.max([Relative_error_2015["2050-Prod"],Relative_error_2015["2050-IMACLIM"]],axis=0)
Relative_error_2015=Relative_error_2015*100
pd.concat([Relative_error_2015.loc[region] for region in Relative_error_2015.index.get_level_values("region").unique()],keys=Relative_error_2015.index.get_level_values("region").unique(),axis=1)

region,AFR,BRA,CAN,CHN,CIS,EUR,IND,JAN,MDE,RAL,RAS,USA
sector,,,,,,,,,,,,
Agriculture,-43.590339,37.134146,15.936325,-44.977011,-79.050224,-7.480474,48.257791,-60.567556,-55.777698,-18.200986,30.372903,-3.244593
Air,-45.142647,-22.186058,12.029989,71.750897,-2.496690,22.254563,46.981273,-44.709538,47.393486,-67.742879,39.151737,-7.363344
Coal,37.069602,43.351854,-88.326434,4.551393,-52.544245,-51.589768,96.477647,-35.200343,100.000000,100.000000,99.602004,-67.145125
Construction,84.523051,99.707779,76.583065,96.611540,71.457156,94.403003,96.048193,72.736079,83.043970,98.905166,96.741961,89.447613
Elec,-39.063432,72.813771,-61.749840,38.125084,-96.819867,-56.260718,27.300270,-32.720686,16.044896,-21.715810,5.708682,-39.344545
Gas,50.168897,81.877660,8.406563,97.880386,4.636883,87.340254,68.934163,100.000000,-13.784912,-2.699496,49.584495,71.617184
Industry,-43.194885,5.480878,-51.361436,21.771290,-72.690944,-30.088759,-9.858828,-60.712734,38.940292,9.302386,11.266783,-54.420385
Liquid fuels,-91.607445,-76.416717,-38.014303,-44.130341,-55.153795,-77.602159,-2.684537,-82.068727,100.000000,100.000000,-61.019781,99.869409
Maritime,-89.620191,-69.921130,46.180283,70.196181,100.000000,41.037293,93.972451,40.129431,-24.836034,-70.236938,-23.554738,-42.914006


In [9]:
Relative_error_2015=production_based_emissions.groupby(["scenario","region"]).sum().loc["baseline"]
Relative_error_2015_ratio=(Relative_error_2015["2015"]-Relative_error_2015["2015-IMACLIM"])/np.max([Relative_error_2015["2015"],Relative_error_2015["2015-IMACLIM"]],axis=0)
Relative_error_2015_ratio=Relative_error_2015_ratio*100
pd.DataFrame(Relative_error_2015_ratio)

,0
region,
AFR,-41.880004
BRA,12.042277
CAN,-21.174507
CHN,42.804422
CIS,-57.997896
EUR,-16.077678
IND,26.399883
JAN,-38.587448
MDE,20.911331


In [10]:
Relative_error_2015["2015"]+model.iot.stressor_extension.F_Y.sum(axis=1,level=0).loc["CO2"]

region
AFR    1.075643e+12
BRA    4.973495e+11
CAN    5.678739e+11
CHN    9.929628e+12
CIS    1.475975e+12
EUR    4.548643e+12
IND    2.118380e+12
JAN    1.628660e+12
MDE    2.189486e+12
RAL    1.286805e+12
RAS    3.212389e+12
USA    5.145479e+12
dtype: float64

In [11]:
import src.advance
final_data_ratio,final_technical_coef,Link_country,Link,Production_volumes,total_consumption,direct_emissions=src.advance.extract_data("IMACLIM_format")

### Comparing Consumption emission

In [12]:
import src.advance
conso=src.advance.exctract_consumption_emission("IMACLIM_format")

/home/ubuntu/miniforge3/envs/MatMat-Trade-env/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [13]:
model_base_year_CO2_conso=model.iot.stressor_extension.F_Y.groupby("region",axis=1).sum().loc["CO2"]
IMACLIM_base_year_CO2_conso=conso.groupby(["Scenario","Region"]).sum().loc["baseline",2015]*1e9
comparison=pd.DataFrame([model_base_year_CO2_conso,IMACLIM_base_year_CO2_conso]).T
comparison.rename(columns={"CO2":"MRIO",2015:"IMACLIM"},inplace=True)
comparison.loc["World","MRIO"]=comparison.loc[:,"MRIO"].fillna(0).sum()
comparison

,MRIO,IMACLIM
AFR,3.122832e+11,3.058000e+11
BRA,8.498662e+10,1.121000e+11
CAN,1.266648e+11,1.297600e+11
CHN,4.574793e+11,5.429000e+11
CIS,2.479844e+11,5.006000e+11
EUR,9.358370e+11,1.360400e+12
IND,2.191543e+11,1.737400e+11
JAN,2.213855e+11,5.126000e+11
MDE,3.301295e+11,3.148000e+11
RAL,2.712905e+11,3.603000e+11


In [14]:
residential=conso.loc[("INDC",slice(None),"Residential"),2015]*1e9
residential.drop("World",level=1,inplace=True)

In [15]:
residential

Scenario  Region  Variable   
INDC      USA     Residential    3.680000e+11
          CAN     Residential    4.539000e+10
          EUR     Residential    5.168000e+11
          JAN     Residential    1.056000e+11
          CIS     Residential    3.436000e+11
          CHN     Residential    4.092000e+11
          IND     Residential    1.220000e+11
          BRA     Residential    2.538000e+10
          MDE     Residential    1.927000e+11
          AFR     Residential    1.121000e+11
          RAS     Residential    1.044000e+11
          RAL     Residential    1.046000e+11
Name: 2015, dtype: float64

### IMACLIM2

In [16]:
imaclim_2=pd.read_excel("../Data/IMACLIM/IMACLIM2.0/IMACLIM_navigate_outputs_NAV_NPi_Default.xlsx",sheet_name="data",index_col=[1,2,3]).drop(columns=["Unit","Model"])

In [17]:
emissions=imaclim_2.loc[pd.Series(imaclim_2.index.get_level_values("Variable"),index=imaclim_2.index).apply(lambda x : "Emissions" in x)].dropna(axis=0)
emissions.reset_index(inplace=True)

In [18]:
emissions.loc[:,"Variable"]=emissions.loc[:,"Variable"].apply(lambda x : x.split('|')[-1])
emissions=emissions.loc[emissions.loc[:,"Variable"].isin(["Electricity","Transportation","Industry","Residential and Commercial","Other Sector"])]
emissions.set_index(["Scenario","Region","Variable"],inplace=True)

In [19]:
emissions

2015      2020  \
Scenario        Region Variable                                         
NAV_NPi_Default USA    Electricity                  1971.00   1668.00   
                       Industry                      506.30    556.80   
                       Residential and Commercial    495.90    526.00   
                       Transportation               1952.00   2058.00   
                       Other Sector                   25.85     24.73   
...                                                     ...       ...   
                World  Electricity                 11750.00  10470.00   
                       Industry                     8078.00   8718.00   
                       Residential and Commercial   2688.00   2912.00   
                       Transportation               7413.00   7635.00   
                       Other Sector                  138.10    146.60   

                                                       2025      2030  \
Scenario        Region Variable                                         
NAV_NPi_Default USA    Electricity                  1583.00   1528.00   
                       Industry                      638.00    692.70   
                       Residential and Commercial    558.00    574.50   
                       Transportation               2130.00   2119.00   
                       Other Sector                   26.92     29.08   
...                                                     ...       ...   
                World  Electricity                  9801.00   9563.00   
                       Industry                    10580.00  11930.00   
                       Residential and Commercial   3178.00   3313.00   
                       Transportation               8566.00   9252.00   
                       Other Sector                  177.60    201.10   

                                                       2035      2040  \
Scenario        Region Variable                                         
NAV_NPi_Default USA    Electricity                  1490.00   1345.00   
                       Industry                      718.90    717.00   
                       Residential and Commercial    578.50    568.10   
                       Transportation               1978.00   1790.00   
                       Other Sector                   30.95     32.12   
...                                                     ...       ...   
                World  Electricity                  9267.00   8967.00   
                       Industry                    12910.00  13690.00   
                       Residential and Commercial   3328.00   3268.00   
                       Transportation               9622.00   9763.00   
                       Other Sector                  218.50    233.60   

                                                       2045      2050  \
Scenario        Region Variable                                         
NAV_NPi_Default USA    Electricity                  1198.00   1099.00   
                       Industry                      716.90    715.10   
                       Residential and Commercial    552.80    539.10   
                       Transportation               1667.00   1631.00   
                       Other Sector                   33.28     34.67   
...                                                     ...       ...   
                World  Electricity                  9071.00   9434.00   
                       Industry                    14320.00  14930.00   
                       Residential and Commercial   3170.00   3105.00   
                       Transportation               9865.00  10050.00   
                       Other Sector                  248.30    261.70   

                                                       2055      2060  \
Scenario        Region Variable                                         
NAV_NPi_Default USA    Electricity                  1028.00    975.40   
                       Industry     

In [21]:
total_per_region=production_based_emissions.groupby(["scenario","region"]).sum().loc["baseline"] 
total_per_region["2015-IMACLIM2"]=emissions.groupby("Region").sum()[2015]*1e9
total_per_region["2050-IMACLIM2"]=emissions.groupby("Region").sum()[2050]*1e9
total_per_region.loc["World",:]=total_per_region.sum(axis=0)
total_per_region[["2015-IMACLIM2","2015","2015-IMACLIM"]]

,2015-IMACLIM2,2015,2015-IMACLIM
region,,,
AFR,1.014076e+12,7.633597e+11,1.313420e+12
BRA,4.820300e+11,4.123629e+11,3.627050e+11
CAN,4.454790e+11,4.412091e+11,5.597290e+11
CHN,9.255410e+12,9.472148e+12,5.417650e+12
CIS,1.939190e+12,1.227990e+12,2.923640e+12
EUR,3.348060e+12,3.612806e+12,4.304940e+12
IND,2.021034e+12,1.899226e+12,1.397832e+12
JAN,2.088313e+12,1.407275e+12,2.291510e+12
MDE,1.725960e+12,1.859356e+12,1.470540e+12


In [22]:
percentage_error=(total_per_region[["2015-IMACLIM2","2015-IMACLIM"]]/total_per_region[["2015"]].values-1)*100
percentage_error

,2015-IMACLIM2,2015-IMACLIM
region,,
AFR,32.843803,72.057822
BRA,16.894615,-12.042277
CAN,0.967762,26.862511
CHN,-2.288164,-42.804422
CIS,57.915741,138.083311
EUR,-7.327978,19.157809
IND,6.413587,-26.399883
JAN,48.394119,62.833162
MDE,-7.174318,-20.911331


In [23]:
percentage_error=(total_per_region[["2050-IMACLIM2","2050-IMACLIM"]]/total_per_region[["2050-Prod"]].values-1)*100
percentage_error

,2050-IMACLIM2,2050-IMACLIM
region,,
AFR,-25.482700,71.042086
BRA,7.859190,-0.915038
CAN,-46.362597,-21.793177
CHN,-31.254595,-43.734347
CIS,40.158633,110.796154
EUR,-9.870533,14.639856
IND,24.198937,-24.096440
JAN,34.308407,57.636492
MDE,-30.550991,-23.375312


In [4]:
from src.utils import get_import_mean_stressor

scenario_list=["INDC","INDC2020_1000","INDC2030_1000","INDC2020_1600","INDC2020_400","INDC2030_1600","baseline"]


NBR_PROCESS=2

start_year=pd.concat([ pd.concat([pd.DataFrame(get_import_mean_stressor(model.iot,region=region),columns=["2015"])  for region in model.regions],
                      keys=model.regions,
                      names=("regions","stressor","sector")) for scenario in scenario_list],
          keys=scenario_list,
          names=("scenario","regions","stressor","sector"))

Emissions=pd.concat([start_year for i in range(4)],axis=1,keys=["2015","2050-all","2050-Prod","2050-emissi"]).sort_index()
Emissions.columns=["2015","2050-all","2050-Prod","2050-emissi"]

for scenario in tqdm.tqdm(range(len(scenario_list))):
    model.new_counterfactual("IMACLIM_emissi_2050_"+scenario, DICT_SCENARIOS["emissivity_IMACLIM"], year=2050,scenario=scenario)
    model.modify_counterfactual("IMACLIM_emissi_2050_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=2050,scenario=scenario,new_name="IMACLIM_all_2050_"+scenario)
    # create the 2050 version based on production changes observed in IMACLIM (since we work on groos output, we need a reference to apply relative change : x_ref), with name new_name
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["production_change_IMACLIM"], year=2050,scenario=scenario,new_name="IMACLIM_all_prod_change_2050_"+scenario,x_ref=model.iot.x.sort_index())
    
    
    ret1=pd.concat([pd.DataFrame(get_import_mean_stressor(model.counterfactuals["IMACLIM_all_2050_"+scenario].iot,region=region),columns=["2050-emiss"])  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    ret2=pd.concat([pd.DataFrame(get_import_mean_stressor(model.counterfactuals["IMACLIM_emissi_2050_"+scenario].iot,region=region),columns=["2050-all"]) for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    ret3=pd.concat([pd.DataFrame(get_import_mean_stressor(model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario].iot,region=region),columns=["2050-Prod"])  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    Emissions.loc[(scenario),"2050-all"],Emissions.loc[(scenario),"2050-emissi"],Emissions.loc[(scenario),"2050-Prod"]=ret1,ret2,ret3
    
    del(model.counterfactuals["IMACLIM_all_2050_"+scenario])
    del(model.counterfactuals["IMACLIM_emissi_2050_"+scenario])
    del(model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario])
    



    scenario=scenario_list[i]
    
     

Problems on sums of columns  region  sector               
AFR     Terrestrial transport     333.209566
MDE     Terrestrial transport    4486.348277
RAL     Terrestrial transport     172.151602
dtype: float64
Problems on sums of columns  region  sector               
AFR     Terrestrial transport     237.323465
MDE     Terrestrial transport    3148.991362
RAL     Terrestrial transport     124.123202
dtype: float64
Problems on sums of columns  region  sector               
AFR     Terrestrial transport     47.043673
MDE     Terrestrial transport    146.430499
RAL     Terrestrial transport     25.875496
dtype: float64
Problems on sums of columns  region  sector               
AFR     Terrestrial transport     169.042045
MDE     Terrestrial transport    2024.383010
RAL     Terrestrial transport      89.719931
dtype: float64
Problems on sums of columns  region  sector               
AFR     Terrestrial transport     282.256477
MDE     Terrestrial transport    6422.740342
RAL     Terrestria

In [ ]:
Emissions

2015          2050  \
                                                         2015          2015   
scenario regions stressor sector                                              
INDC     AFR     CH4      Agriculture            1.652303e+06  1.652303e+06   
                          Air                    1.707681e+06  1.707681e+06   
                          Coal                   6.282049e+05  6.282049e+05   
                          Construction           2.464265e+06  2.464265e+06   
                          Elec                   1.198899e+06  1.198899e+06   
...                                                       ...           ...   
baseline USA     SF6      Liquid fuels           2.694805e+03  2.694805e+03   
                          Maritime               3.486621e+02  3.486621e+02   
                          Oil                    3.329066e+01  3.329066e+01   
                          Services               1.103385e+03  1.103385e+03   
                          Terrestrial transport  7.582619e+02  7.582619e+02   

                                                    2050-Prod     2050-Cons  
                                                         2015          2015  
scenario regions stressor sector                                             
INDC     AFR     CH4      Agriculture            1.652303e+06  1.652303e+06  
                          Air                    1.707681e+06  1.707681e+06  
                          Coal                   6.282049e+05  6.282049e+05  
                          Construction           2.464265e+06  2.464265e+06  
                          Elec                   1.198899e+06  1.198899e+06  
...                                                       ...           ...  
baseline USA     SF6      Liquid fuels           2.694805e+03  2.694805e+03  
                          Maritime               3.486621e+02  3.486621e+02  
                          Oil                    3.329066e+01  3.329066e+01  
                          Services               1.103385e+03  1.103385e+03  
                          Terrestrial transport  7.582619e+02  7.582619e+02  

[6048 rows x 4 columns]

In [ ]:
from src.utils import get_import_mean_stressor

NBR_PROCESS=4

scenario_list=["INDC","INDC2020_1000","INDC2030_1000","INDC2020_1600","INDC2020_400","INDC2030_1600","baseline"]

start_year=pd.concat([ pd.concat([pd.DataFrame(footprint_extractor(model,region=region),index=[2015]).T  for region in model.regions],
                      keys=model.regions,
                      names=("regions","types")) for scenario in scenario_list],
          keys=scenario_list,
          names=("scenario","regions","types"))

Emissions=pd.concat([start_year for i in range(4)],axis=1,keys=["2015","2050","2050-Prod","2050-Cons"]).sort_index()
Emissions.columns=["2015","2050","2050-Prod","2050-Cons"]

production_based_emissions=pd. concat([ pd.concat([ model.iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"]for i in range(4)],axis=1,keys=["2015","2050","2050-Prod","2050-Cons"]) for scenario in scenario_list],
                                        keys=scenario_list,
                                        names=("scenario","region","sector"),
                                        axis=0).sort_index()


def task(scenario):
    model.new_counterfactual("IMACLIM_all_2050_"+scenario, DICT_SCENARIOS["emissivity_IMACLIM"], year=2050,scenario=scenario)
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=2050,scenario=scenario)
    # create the 2050 version based on production changes observed in IMACLIM (since we work on groos output, we need a reference to apply relative change : x_ref), with name new_name
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["production_change_IMACLIM"], year=2050,scenario=scenario,new_name="IMACLIM_all_prod_change_2050_"+scenario,x_ref=model.iot.x.sort_index())
    
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["consumption_change_imaclim"], year=2050,scenario=scenario,new_name="IMACLIM_all_cons_change_2050_"+scenario)
    
    ret1=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    ret2=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    ret3=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    ret4=model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    ret5=model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    ret6=model.counterfactuals["IMACLIM_all_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    del(model.counterfactuals["IMACLIM_all_2050_"+scenario])
    del(model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario])
    del(model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario])
    
    return ret1,ret2,ret3,ret4,ret5,ret6


pool = multiprocessing.Pool(processes=NBR_PROCESS)
result_list=pool.map(task,scenario_list)

for i in range(len(scenario_list)):
    scenario=scenario_list[i]
    Emissions.loc[(scenario),"2050"],Emissions.loc[(scenario),"2050-Prod"],Emissions.loc[(scenario),"2050-Cons"],production_based_emissions.loc[(scenario),"2050-Cons"],production_based_emissions.loc[(scenario),"2050-Prod"],production_based_emissions.loc[(scenario),"2050"]=result_list[i]
        
    